# Overview

This tutorial is going to introduce some simple tools for detecting sentiment in Tweets. We will be using a set of tools called the Natural Language Toolkit ([NLTK](http://www.nltk.org)). This is collection of software written in the [Python](https://www.python.org) programming language. An important design goal behind Python is that it should be easy to read and fun to use, so well-suited for beginners. A similar motivation inspired NLTK: it should make complex tasks easy to carry out, and it should be written in a way that would allow users to inspect and understand the code.

Why is this relevant? Well, a lot of software these days is built to be easy to use, but hard to inspect. For example, smartphones have a lot of slick apps on them, but very few people have the expertise to look under the wrappiing to find out how they work.

This tutorial is written using the [Ipython](http://ipython.org) framework. This allows text to be interspersed by fragments of code, occuring in special "cells". Just below is a cell where we are using Python to do a simple calculation:

In [1]:
3 + 4

7

Some of the cells will contain snippets of code that are necessary for the big story to work, but which you don't need to understand. We'll try to make it clear when it's important for you to pay attention to one of the cells. 

In [4]:
import nltk

In [5]:
from nltk.corpus import twitter_samples

In [6]:
input_file = twitter_samples.abspath("tweets.20150430-223406.json")

In [8]:
from nltk.twitter.util import json2csv
with open(input_file) as fp:
    json2csv(fp, 'tweets_text.csv', ['text'])

In [9]:
ls

Twitter sentiment analysis.ipynb  tweets_text.csv


In [12]:
import pandas as pd
tweets = pd.read_csv('tweets_text.csv', header=0, encoding="utf8")
tweets.head(20)

,text
0,RT @KirkKus: Indirect cost of the UK being in ...
1,VIDEO: Sturgeon on post-election deals http://...
2,RT @LabourEoin: The economy was growing 3 time...
3,RT @GregLauder: the UKIP east lothian candidat...
4,RT @thesundaypeople: UKIP's housing spokesman ...
5,RT @Nigel_Farage: Make sure you tune in to #As...
6,RT @joannetallis: Ed Milliband is an embarrass...
7,RT @abstex: The FT is backing the Tories. On a...
8,RT @NivenJ1: “@George_Osborne: Ed Miliband pro...
9,LOLZ to Trickle Down Wealth. It's never trickl...


In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer